# Resumimos las Foto Previsión, de forma que tnegamos un % de ocupación por Hotel, Fecha Foto y Previsión

_Importación de librerias_

In [1]:
import pandas as pd
import tfm_helpers as tfm

pd.set_option("display.max_columns", None)

Procesamos los ficheros para generar previsiones agrupadas por año

In [2]:
years_to_process = ["2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]
# years_to_process = ["2006"]
resumen_ocupacion_data = pd.DataFrame()

for year in years_to_process:
    file = tfm.getPathToProcessedSnapshotOcupacion("parquet", year)

    current_ocupacion_data = pd.read_parquet(file, columns=["Hotel", "Fecha foto", "Fecha prevision", "Cantidad habitaciones"])
    current_ocupacion_data["Hotel"] = current_ocupacion_data.Hotel.astype("category")
    current_ocupacion_data.set_index(["Hotel", "Fecha foto", "Fecha prevision"], inplace=True)
    current_ocupacion_data = pd.DataFrame(current_ocupacion_data.groupby(["Hotel", "Fecha foto", "Fecha prevision"])["Cantidad habitaciones"].agg("sum"))
    filtro_habitaciones = (current_ocupacion_data["Cantidad habitaciones"]>0)
    resumen_ocupacion_data = pd.concat([resumen_ocupacion_data, current_ocupacion_data[filtro_habitaciones]])

resumen_ocupacion_data.to_parquet(tfm.getPathToProcessedSnapshotResumenOcupacion("parquet"), index=True)


In [3]:
df = resumen_ocupacion_data
df.head()

Cantidad habitaciones
Hotel Fecha foto Fecha prevision                       
04    2006-01-01 2006-02-12                           1
                 2006-02-13                           1
                 2006-02-14                           1
                 2006-02-15                           1
                 2006-02-16                           1